# <span style="color:teal;">CIS 211 Project 8:  &nbsp; Bears and Fish</span>

##### Due 11:00 P.M. Thursday Mar 16

##### <span style="color:red">Group Members:</span>

Group work is allowed for this project, but groups are exepcted to do more work (see markdown cells at the end of the notebook for suggestions and ideas).  

If you work in a group only one group member should upload this notebook to Canvas.  Edit this cell to include the name and DuckID for each member of the group:

**Name:**

**Name:**

**Name:**

##  <span style="color:teal;">Overview</span> 

The last project this term is a cellular automaton style biological simulation.  

The sytem is described in Chapter 11 of the Miller and Ranum text.  **Read the description of the simulation in the textbook before you start working on this project.**  Your code will be quite a bit different than the code in the book, but the rules of the simulation are the same, and you need to know how objects interact.

The project has three main parts:
* A class named World that defines a 2D grid where cells are accessed according to their row and column coordinates
* Classes for Bear and Fish, the organisms that can inhabit the world
* Two top level functions, `wbf` and `step_system`; the first will make a world and populate it at random with bears and fish, and the second will run a simulation using that world.

We're written `step_system` for you -- it's in the last code cell in this notebook -- but you need to design, implement, and test the classes and the `wbf` function.

##  <span style="color:teal;">Libraries</span> 

Your program will need to use `numpy` (a numeric processing library) and `random` (the builtin random number generator module).  You can import additional modules if you wish; add the import statements to this code cell.

In [1]:
import numpy as np
import random

##  <span style="color:teal;">Event Log</span> 

A useful debugging technique is to save descriptions of events in a log.  Any code in the simulator, whether it is the top level function or a method in one of your classes, can call a function named `log`, passing it a description of an event that just occurred.  Examples might be 
```
new Fish in cell (x,y)
```
or 
```
Bear in cell (x,y) eats Fish in cell (x,y)
```

The function below uses a global variable named `logging`.  If you want to take advantage of the `log` function:
* edit the cell to set `logging` to True
* add calls to `log` at various points in your methods

Later, when doing large scale simulations, you can turn off logging simply by setting `logging` to False before you call the top level simulation function.

**Note:** &nbsp; you can change the definition of the `log` function however you want, _e.g._ you can have it write log messages to a file or printadditional information.

In [2]:
logging = False

def log(message):
    if logging:
        print(message)

##  <span style="color:teal;">World (20 points)</span>

Describe your World class in the following markdown cell, and write the code for the class in the code cell below the markdown cell.

The minimum requiremens for this class are:
* the constructor should be passed the grid size (number of rows and columns)
* all cells should contain `None` when the world is initialized; later they can contain references to Fish or Bear objects
* include a method named `biota` that returns a list of Fish and Bear objects currently in the grid

You should also define the methods that allow you to access a cell or store a value in a cell using Python's indexing operator.  For example, if `w` is a World object, `w[i,j]` should return the object in row `i`, column `j` (which could be None).  You should also be able to store an item in the grid by using `w[i,j]` on the left side of an assignment statement.

You can have additional instance methods, class variables, or class methods.  Make sure you describe any new additions in the documentation.

**Note:** &nbsp; There are no auto-grader tests for the World class; tests for this class will be done as part of the tests for the Bear and Fish classes.

##### <span style=color:red>Documentation</span> 

**Important** Write your documentation in the following markdown cell.  Do not delete or move this cell.

x and y return the array coordinates for compuations in animal.adjacent
simple coordinate check to see if a space is filled up
insert puts an animal in its own coordinates, its pretty clever because it uses animal's attributes to put it inside, no need fo an extra tuple input
delete just makes a space into none (sorta like the death star and alderaan)
who checks the spot and returns what it is
represent gives the array that 'self._grid' makes
and biota does what we want it to


##### <span style=color:red>Code</span> 

**Important** Write the definition of your World class in the following code cell. Do not delete or move this cell.

In [2]:
class World():
    def __init__(self, rows, cols, fish=0, bears=0):
        self._rows = rows
        self._cols = cols
        self._fish = fish
        self._bears = bears
        self._grid = np.array([None] * (self._rows * self._cols)).reshape(self._rows, self._cols)
            
    def x(self):
        return self._rows -1
    
    def y(self):
        return self._cols -1
    
    def check_coord(self, tup):
        if (0 <= tup[0] <= self._rows-1):
            if (0 <= tup[1] <= self._cols-1):
                return True
        return False
            
    def insert(self, animal):
        tup = animal.location()
        if self.check_coord(tup):
            self._grid[tup] = animal
            
            
    def delete(self, tup):
        if self.check_coord(tup):
            self._grid[tup[0], tup[1]] = None
            
        
    def who(self, tup):
        if self.check_coord(tup):
            return self._grid[tup[0], tup[1]]
    
    def represent(self):
        return self._grid 
    
    def biota(self):
        res_list = list()
        for col in list(range(self._cols)):
            for row in list(range(self._rows)):
                if self._grid[col, row] != None:
                    i_tup = self._grid[col, row]
                    res_list.append(i_tup)
        return res_list
        

## <span style="color:teal;">Fish and Bears</span>

To run a simulation we need to add a random collection of animals to the world.  The two types of animals in this simulation are fish and bears, and you will write class definitions named Fish and Bear that implement the behaviors of the animals.

### Fish Class

During the simulation a Fish object needs to behave as follows:

(1) Fish are susceptible to overcrowding:  if there are fish in 2 or more neighboring cells the fish dies (it's removed from the simulation)

(2) A fish can reproduce if it has been alive for a certain number of time steps: a random neighboring cell is chosen, and if that cell is empty, a new fish is placed in that cell

(3) A fish can move to another cell:  it picks a random direction, and if the neighboring cell in that direction is unoccupied the fish moves there

The constructor for the Fish class will be passed a reference to a World object and a location, in the form of a tuple with a row number and column number (the object needs to know its location so it can look for other objects in neighboring cells).

The class should include the following methods:
* `live` implements rules 1 and 2 shown above
* `move` implements rule 3
* `location` returns the current grid location (row and column) of the object

Define a class variable named `breed_interval` to specify how many time steps a fish must be alive before it reproduces; the initial value for this variable is 12.


### Bear Class

During the simulation a Bear object needs to behave as follows:

(1) A bear looks for fish in each adjacent cell; if it finds one or more fish it eats one at random 

(2) If a bear has not eaten for certain number of time steps it dies (it's removed from the simulation)

(3) A bear can reproduce if it has been alive for a certain number of time steps: a random neighboring cell is chosen, and if that cell is empty, a new bear is placed in that cell

(4) A bear can move to another cell:  it picks a random direction, and if the neighboring cell in that direction is unoccupied the bear moves there

Define a class variable named `breed_interval` to specify how many time steps a bear must be alive before it reproduces; the initial value for this variable is 8.  Define another class variable named `survive_without_food` to be the number of time steps a bear can live before it dies from starvation; the initial value for this variable is 10.

The constructor will be passed a reference to a World object and a location, in the form of a tuple with a row number and column number (the object needs to know its location so it can look for other objects in neighboring cells).

The class should include the following methods:
* `live` implements rules 1, 2, and 3 shown above
* `move` implements rule 4
* `location` returns the current grid location (row and column) of the object

### Animal (Base Class)?

From the descriptions above it should be apparent that fish and bears have some things in common.

For **full credit** on the coding and documnetation portions of this project you should define a class named Animal and use it as the base class for your Fish and Bear classes.  Some things to think about as you design your classes:
* are there behaviors or operations that are common to both and that can be implemented just once in Animal?
* perhaps a behavior or operation can be defined with a default in the Animal class, and then overridden in the derived class?

One strategy you might consider is to write a complete implementation for one class, either Bear or Fish.  After you have debugged the class and it passes its unit tests you'll have a better idea of what to implement in the other class.  Then you can start moving common behaviors to the Animal class while you are writing the other derived class.

You can still receive **partial credit** if you skip the Animal class and simply write completely separate Fish and Bear classes.  None of the unit tests assume there is a class named Animal.

### Details and Hints

**Fish:** &nbsp; The way the simulation is defined a fish might be eaten before the top level simulation calls the `live` method.  Your `live` method should check to make sure the fish is still alive.  The easiest way to do this is to include an instance variable named `_alive` that is set to True when the fish is initialized and set to False when it dies.

You can have additional instance methods, class variables, or class methods.  Make sure you describe any new additions in the documentation.

**Animal:** &nbsp; Keep this in mind when you design the class hierarchy: every object has an attribute named ``__class__`` (with two underscores before and after the name).  It is a reference to the class an object was defined with.  As an example of how to use it, consider what would happen if we want to define `reproduce` in the Animal class so it is inherited by both Fish and Bear classes.  We need to know the value of `breed_interval` in each object's own class.  We can find this value using the expression
```
self.__class__.breed_interval
```
This will be a reference to `Fish.breed_interval` or `Bear.breed_interval`, depending on whether a `reproduce` was called with Fish object or Bear object.


### <span style="color:teal;">Animal (20 points)</span>

##### <span style=color:red>Documentation</span> 

If you implement the Animal class write your documentation in the following markdown cell.  

**Important** Do not delete or move this cell.

The animal age is how many 'turns' old the animal is
emoji is just the nice way that ill display the other animals
age returns how many turns old the animal is
birthday throws the animal a little birthday party
x returns the x value of the coordinates
y does the same for y
location does what x and y do but returns a tuple
new_pos just updates an animals coordiantes
adjacent returns a list of nearby spaces of an animal, also implements the torus effect cause we had to do that
and __repr__ is an emoji value of None, because we dont know what emoji to put, but when inherited it will be different

##### <span style=color:red>Code</span> 

If you implement the Animal class write the definition in the following code cell

**Important:** &nbsp; Do not delete or move this cell.

In [3]:
class Animal():
    def __init__(self, world, tup_loc, emoji=None, age=0):
        self._emoji = emoji
        self._tup_loc = tup_loc
        self._world = world
        self._age  = age
        self.breed_interval = None
        
    def age(self):
        return self._age
    
    def birthday(self):
        self._age += 1
        
    def x(self):
        return self._p_row
    
    def y(self):
        return self._p_col
    
    def location(self):
        return self._tup_loc
    
    def new_pos(self, new_tup):
        self._tup_loc = new_tup
 
    def __repr__(self):
        return '{}'.format(self._emoji)
                        
    def adjacent(self):
        world = self._world
        x = self._tup_loc[0]
        y = self._tup_loc[1]
        neighbor_spaces = list( ((x - 1, y - 1), (x, y - 1), (x + 1, y - 1),
                            (x - 1, y),     (x + 1, y),
                            (x - 1, y + 1), (x, y + 1), (x + 1, y + 1) ) )
        nl = list()
        
        for coord in list(range(len(neighbor_spaces))): #tourus effect
        
            neighbor_spaces[coord] = list(neighbor_spaces[coord])#tuples are immutable, must change to a list
            
            x = neighbor_spaces[coord][0]
            y = neighbor_spaces[coord][1]

            if x < 0:
                x += world.x()+1
            if y < 0:
                y += world.y()+1
            if x > world.x():
                x -= world.x()-1
            if y > world.x():
                y -= world.x()-1

            neighbor_spaces[coord] = (x,y) #i want to return a list of tuples
            nl.append((x,y))
        
        
        return tuple(nl)

        
        

### <span style="color:teal;">Fish (20 points)</span>

##### <span style=color:red>Documentation</span> 

**Important:** &nbsp; Write the documentation for your Fish class in the following markdown cell.  Do not delete or move this cell.

here we make a subclass of animal, we give fish the ability to move and reproduce, live checks for overcrowding and deletes accordingly, and checks if the fish is able to reproduce, if it is then it makes a fish baby to explore

and inser inserts the fish into the grid properly and breed interval is 12 as required

we have to make reproduce in fish because we have to implement its class specific offspring

##### <span style=color:red>Code</span> 

**Important:** &nbsp; Write the Python code for your Fish class in the following code cell.  Do not delete or move this cell.

In [4]:
class Fish(Animal):
    breed_interval = 12

    def __init__(self, world, tup_loc, age=0):
        Animal.__init__(self, world, tup_loc)
        self._tup_loc = tup_loc
        self._age = age
        self._world = world
        self._emoji = "\U0001F41F"
        self.inser()
    
    def inser(self):
        self._world.insert(self)
        
    def move(self):
        near = self.adjacent()
        roll = random.randint(0, 7)

        new_space = near[roll]
        
        n_age = self.age()
        if self._world.who(new_space) == None:
            self._world.delete(self.location())
            self.new_pos(new_space)
            self._world.insert(Fish(self._world, new_space, age=n_age))
        
    def reproduce(self):
        pob = self.adjacent()
        l = random.randint(0,6)
        self._age = 0
        self._world.insert(Fish(self._world,pob[l]))
        
                
    def live(self):

        adj = self.adjacent()
        count=0
        self.birthday()

        for space in adj:
            t = self._world.who(space)
            if isinstance(t, Fish):
                count += 1
            if count >= 2:    
                self._world.delete(space)
                
        if self.age() == Fish.breed_interval:
            self.reproduce()
        

##### <span style="color:red">Autograder Tests:</span>


**Important:** &nbsp;  the code cells in this section will be used by `nbgrader` to run automated tests.  Do not move, delete or alter these cells in any way.

In [53]:
# A new world has no objects
w1 = World(5,5)
assert len(w1.biota()) == 0

# After adding a fish there should be one object
f1 = Fish(w1, (2,2))
assert len(w1.biota()) == 1

# Test the location method
assert f1.location() == (2,2)

In [54]:
# Setting breed_interval to 1 should cause a fish to reproduce when live is called
w2 = World(5,5)
Fish.breed_interval = 1
f2 = Fish(w2, (2,2))
f2.live()
assert len(w2.biota()) == 2

# Reset the interval to original value for remaining tests
Fish.breed_interval = 12

In [55]:
# Make three fish, the one in the middle should die from overcrowding
w3 = World(5,5)
f3 = Fish(w3, (2,2))
Fish(w3, (1,1))
Fish(w3, (3,3))
f3.live()
assert len(w2.biota()) == 2

In [56]:
# When a fish moves it should be within one cell of its original location
w4 = World(5,5)
f4 = Fish(w4, (2,2))
f4.move()
r, c = f4.location()
assert (r,c) != (2,2)
assert abs(r-2) <= 1 and abs(c-2) <= 1

###  <span style="color:teal;">Bear (20 points)</span>

##### <span style=color:red>Documentation</span> 

**Important:** &nbsp; Write the documentation for your Bear class in the following markdown cell.  Do not delete or move this cell.

The bear class is almost the same as the fish class but it checks for how long the bear has been fasting, or 'fast'
fast birthday indicates the 'birthday' of the fast time, much like the original birthday method works, just custom for bears
we set the emoji so we can properly __repr__
eat checks all nearby spaces for fish to eat, and a pass is put at the end so the bear cant pig out
again, we have to make another reproduce because we have to implement its class specific offspring

##### <span style=color:red>Code</span> 

**Important:** &nbsp; Write the Python code for your Bear class in the following code cell.  Do not delete or move this cell.

In [5]:
class Bear(Animal):
    breed_interval = 8
    survive_without_food = 10

    def __init__(self, world, tup_loc, age=0, fast=0):
        Animal.__init__(self, world, tup_loc)
        self._tup_loc = tup_loc
        self._age = age
        self._world = world
        self._fast = fast
        self._emoji = "\U0001F43B"
        self.inser()
        
    def fast_birthday(self):
        self._fast += 1
        
    def inser(self):
        self._world.insert(self)
        
    def move(self):
        near = self.adjacent()
        roll = random.randint(0, 7)

        new_space = near[roll]
        
        n_age = self.age()
        n_fast = self._fast
        if self._world.who(new_space) == None:
            self._world.delete(self.location())
            self.new_pos(new_space)
            self._world.insert(Bear(self._world, new_space, age=n_age, fast=n_fast))
        
    def reproduce(self):
        pob = self.adjacent()
        l = random.randint(0,6)
        self._age = 0
        self._world.insert(Bear(self._world,pob[l]))
        
    def eat(self):
        adj = self.adjacent()
        for space in adj:
            t = self._world.who(space)
            if isinstance(t, Fish):
                self._world.delete(space)
                self._fast = 0
                pass
            
    def live(self):
        
        if self._fast == Bear.survive_without_food:
            
            tup = self.location()
            self._world.delete(tup)
            
        adj = self.adjacent()
        count=0
        self.birthday()
        self.fast_birthday()
        self.eat()
        
        if self.age() == Bear.breed_interval:
            self.reproduce()

##### <span style="color:red">Autograder Tests:</span>

**Important:** &nbsp;  the code cells in this section will be used by `nbgrader` to run automated tests.  Do not move, delete or alter these cells in any way.

In [58]:
# Test the Bear constructor and location method
w1 = World(5,5)
b1 = Bear(w1, (1,1))
assert len(w1.biota()) == 1
assert b1.location() == (1,1)

In [59]:
# Repeat the reproduction test for Bears
w2 = World(5,5)
Bear.breed_interval = 1
b2 = Bear(w2, (2,2))
b2.live()
assert len(w2.biota()) == 2
Bear.breed_interval = 8

In [60]:
# Make fish for the bear to eat, count the number of objects after eating
w3 = World(5,5)
b3 = Bear(w3, (2,2))
Fish(w3, (1,1))
Fish(w3, (3,3))
b3.live()
assert len(w2.biota()) == 2

In [61]:
# Setting the survival limit to 1 should cause a Bear to starve 
w4 = World(5,5)
Bear.survive_without_food = 1
b4 = Bear(w4, (2,2))
b4.live()
assert len(w4.biota()) == 0

AssertionError: 

In [77]:
Bear.survive_without_food = 10

In [78]:
# Repeat the move test for bears
w5 = World(5,5)
b5 = Bear(w5, (2,2))
b5.move()
r, c = b5.location()
assert (r,c) != (2,2)
assert abs(r-2) <= 1 and abs(c-2) <= 1

##  <span style="color:teal;">The `wbf` Function (10 points)</span>

Fill in the body of the `wbf` function so it returns a new World object with the specified number of rows and columns and with the specified number of Bear and Fish objects at random locations.

When we grade your project we will call `wbf` to make a World object and then use the main loop (implemented by `step_system`) to run the simulation.

**There is no documentation requirement for the `wbf` function.**

In [6]:
def wbf(nrows, ncols, nbears, nfish):
    '''
    Return a new World object with the specified number of Bear and Fish objects.
    '''
    w = World(nrows, ncols)
    
    for animal in range(nfish):
        x = random.randint(0, nrows-1)
        y = random.randint(0, ncols-1)
        tup = x,y
        if w.who(tup) == None: #checks if the square is empty
            w.insert(Fish(w,tup))
        else: #if the square is not1, puts the animal in an adjacent spcae
            c = random.randint(-1, 0)
            v = random.randint(-1, 0)
            tup = (x+c,y+v)
            w.insert(Fish(w,tup))
                
    for animal in range(nbears):
        x = random.randint(0, nrows-1)
        y = random.randint(0, ncols-1)
        tup = x,y
        if w.who(tup) == None: #checks if the square is empty
            w.insert(Bear(w,tup))
        else: #if the square is not, puts the animal in an adjacent spcae
            c = random.randint(-1, 0)
            v = random.randint(-1, 0)
            tup = (x+c,y+v)
            w.insert(Bear(w,tup))
        
    if len(w.biota()) != (nfish + nbears): #fixed an ednge case where griddit would do wrong things
        wbf(nrows, ncols, nbears, nfish)
        
    return w

##### <span style="color:red">Autograder Tests:</span>

**Important:** &nbsp;  the code cells in this section will be used by `nbgrader` to run automated tests.  Do not move, delete or alter these cells in any way.

In [80]:
w = wbf(10,10,3,12)

dct = { Bear: 0, Fish: 0 }
for x in w.biota():
    dct[x.__class__] += 1

assert dct[Bear] == 3
assert dct[Fish] == 12

##  <span style="color:teal;">The `step_system` Function</span>

We've written this function for you.  It will run a single time step of the simulation.  Pass it a World object containing a grid populated with Bear and Fish objects and it will (a) see which animals survive, then (b) allow all the animals to move to a new location.

In [7]:
def step_system(world):
    for x in world.biota():
        x.live()
    for x in world.biota():
        x.move()

In [12]:
x = wbf(10, 10, 5, 7)
print(x.represent(), '\n')
for i in range(5):
    step_system(x)
print(x.represent())
print(x.biota())

[[None 🐟 None 🐟 None None None None None None]
 [None None None None None None None None None None]
 [None None None None None None None None None None]
 [None None None 🐟 None None None None None None]
 [None None None None None None None None None None]
 [None None None None None 🐟 None 🐻 None 🐟]
 [None 🐻 None None None None None None None None]
 [None None None 🐟 None None None None 🐻 None]
 [None None None None 🐟 None None None None None]
 [None None None None None None 🐻 None None None]] 

[[None None None None None None None None None None]
 [None None None None 🐟 None None 🐻 None None]
 [None None 🐟 None None None None None None 🐻]
 [None None None None None None None None None None]
 [None None None None None None None 🐟 None None]
 [None None None None None None None None None None]
 [None None None 🐻 None None None None None None]
 [None None None None None None None None None None]
 [None 🐻 None 🐟 None None None None None None]
 [None None None None None None None None None 

##  <span style="color:teal;">Experiments (10 points)</span>

Run some experiments with the top level simulation loop and describe the results in the markdown cell below.  Some ideas of things to try:
* The settings for the Bear class `breed_interval` and `survive_without_food` variables come from the textbook.  Will the world ever run out of bears with these settings?
* Change the settings so `Bear.breed_interval` is larger than `Bear.survive_without_food`.  How does that change the outcome?
* Set the `breed_interval` counter for the Fish class to a smaller number (e.g. 4) so the world has more fish.  What effect does that have?
* Write a function that runs the simulation for a specified number of generations, or until there are no more objects left in the grid.  What combination of parameters leads to the largest number of time steps before the simulation halts?

YOUR DOCUMENTATION HERE

##  <span style="color:teal;">Projects for Groups and/or Extra Credit</span>

Here are some suggestions for ways to extend the simulation.  We will consider other types of extensions -- send a request to `conery@uoregon.edu` with your proposal.

Groups with three people should implement two extensions.  Groups with two people can choose either extension, or do both for extra credit.

* Implement the Plant class described in Section 10.7 of the textbook, and modify the Fish class so fish eat plants and die if they don't find enough food.

* If you implement Plant, how does that affect the class hierarchy?  Is there an even more general class called Organism, with Plants and Animals subtypes of organism?

* Experiment with data structures: make a second version of the World class, but use a list-of-lists approach to making the grid.  Which is more efficient, a list of lists or a numpy array?  Which scales better when larger worlds are used in the simulation?

* Implement a GUI using `tkinter` that is similar to the Solar System GUI.  A canvas should display the world along with images for the Bear and Fish objects (you can download `Bear.gif` and `Fish.gif` from Canvas).  Use a spinbox or text entry box to specify the number of Bear and Fish objects and the number of time steps to run.  Include a Run button to start the simulation.
